# take sample from 5, and gen wdls

In [ ]:
sys.exit()
import h5py
# f.close()
fs.close()
fd.close()
# ft.close()
fs = h5py.File('AllStates_intVec.hdf5', 'r')
# fd = h5py.File('5PPpKk.hdf5', 'w')
fd = h5py.File('5PPpKk.hdf5', 'a')

# fs.copy('5PPpKk', fd['/'])
fdn = list(fd)
print(fdn)

x = fd[fdn[0]]

    

lFull = len(x)
print(lFull)
lPart = 500000
# lPart = lFull//1000
# print(lPart)
# fd.create_dataset('5PPpKk_random500k', shape=(500000,5))
dd = fd['5PPpKk_random500k']

# import sys
# sys.exit()

In [ ]:
sys.exit()
from random import randint
for i in range(500):
    r = randint(0,lFull - 1000)
    xPart = x[r:r+1000]
    dd[i*1000:i*1000+1000] = xPart 
    
# f = h5py.File('5PPpKk_TlTest.hdf5', 'a')
# fn = list(f)
# fn
# f.create_dataset('5PPpKk_0.01')


### copy dataset

In [ ]:
sys.exit()
fd.close()
with h5py.File('5PPpKk.hdf5', 'a') as fs:
    fsn = list(fs)
    print(fsn)
    ds = fs[fsn[1]]
    
    with h5py.File('5PPpKk_random500k.hdf5', 'w') as fd:
        fs.copy(fsn[1], fd['/'])
        fdn = list(fd)
        print(fdn)
